In [105]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy as sp
from scipy import ndimage
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.model_selection import train_test_split

from scipy import sparse
from IPython.display import Image

from mpl_toolkits.mplot3d import Axes3D

sns.set(style="ticks", color_codes=True)
%matplotlib inline

import os
import json

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from hyperopt import hp
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

from datetime import datetime
import dateutil

from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier

#For re-sizing
from skimage.transform import resize
from scipy import misc
import matplotlib.pyplot as plt
import skimage

import mdptoolbox.example
import cvlib as cv
from matplotlib.pyplot import imshow
from PIL import Image, ImageStat
import cv2
from cvlib.object_detection import draw_bbox

import math

In [104]:
def image_colorfulness(image):
    # split the image into its respective RGB components
    (B, G, R) = cv2.split(image.astype("float"))
 
    # compute rg = R - G
    rg = np.absolute(R - G)
 
    # compute yb = 0.5 * (R + G) - B
    yb = np.absolute(0.5 * (R + G) - B)
 
    # compute the mean and standard deviation of both `rg` and `yb`
    (rbMean, rbStd) = (np.mean(rg), np.std(rg))
    (ybMean, ybStd) = (np.mean(yb), np.std(yb))
 
    # combine the mean and standard deviations
    stdRoot = np.sqrt((rbStd ** 2) + (ybStd ** 2))
    meanRoot = np.sqrt((rbMean ** 2) + (ybMean ** 2))
 
    # derive the "colorfulness" metric and return it
    return stdRoot + (0.3 * meanRoot)

def percieved_brightness(im_file):
   im = Image.open(im_file)
   stat = ImageStat.Stat(im)
   r,g,b = stat.mean
   return math.sqrt(0.241*(r**2) + 0.691*(g**2) + 0.068*(b**2))

def brightness(im_file):
   im = Image.open(im_file).convert('L')
   stat = ImageStat.Stat(im)
   return stat.mean[0]

def detect_genders(face, conf, img_filename):
    # apply face detection
    img = cv2.imread(img_filename)
    #face, conf = cv.detect_face(img)
    gender_array = []
    # loop through detected faces
    for f in face:

        (startX,startY) = f[0],f[1]
        (endX,endY) = f[2],f[3]

        # draw rectangle over face
        cv2.rectangle(img, (startX,startY), (endX,endY), (0,255,0), 2)

        face_crop = np.copy(img[startY:endY, startX:endX])

        # apply gender detection
        (label, confidence) = cv.detect_gender(face_crop)

        #print(confidence)
        #print(label)

        idx = np.argmax(confidence)
        label = label[idx]
        gender_array.append(label)
    return gender_array


In [89]:
paths = ["alexachung", "bucketlistjourney", "mariekondo", "jamieoliver", "gypsea_lust", 
         "kimkardashian", "tombrady", "ocasio2018"]
# filelist = os.listdir(path)
followers = {}
followers["alexachung"] = float(3300000)
followers["bucketlistjourney"] = float(100000)
followers["mariekondo"] = float(3000000)
followers["jamieoliver"] = float(6700000)
followers["gypsea_lust"] = float(2100000)
followers["kimkardashian"] = float(138000000)
followers["tombrady"] = float(6100000)
followers["ocasio2018"] = float(3400000)

df = pd.DataFrame(columns=['filename', 'num_faces', 'percieved_brightness', 
                           'colorfulness', 'people', 'brightness', 'people', 'likes' ])

In [90]:
image_arrays = []
for p in paths:
    filelist = os.listdir(p)
    for file in filelist:
        if str(file)[-1] != "g":
            continue
        file_name = p + str("/") + str(file)
        pic = load_img(file_name)
        arr = img_to_array(pic)
        image_arrays.append((p, str(file), arr))
        df = df.append({'filename' : file_name} , ignore_index=True)

In [91]:
dataDict = {}
for path in paths:
    with open(path + str("/") + path + str(".json")) as f:
        data = json.load(f)
    dataDict[path] = data

likes = {}
final_arr = []
likes_array = []
for img in image_arrays:
    jsonData = dataDict[img[0]]
    for i in range(len(jsonData['GraphImages'])):
        if img[1] in jsonData['GraphImages'][i]["display_url"]:
            
            image_file_name = img[0] + str("/") + img[1] 
            
            image = Image.open(image_file_name, 'r')
            image_array = img_to_array(pic)
            image_cv2 = cv2.imread(image_file_name)

            # Add likes to the df
            likes[img[1]] = jsonData['GraphImages'][i]["edge_media_preview_like"]["count"]
            df.loc[df['filename'] == image_file_name,'likes'] = jsonData['GraphImages'][i]["edge_media_preview_like"]["count"]/float(followers[img[0]])
            
            # Add number of faces 
            faces, confidences = cv.detect_face(image_cv2)
            df.loc[df['filename'] == image_file_name, 'num_faces'] = len(confidences)

            # Add objects
            bbox, label, conf = cv.detect_common_objects(image_cv2)
            #df.loc[df['filename'] == image_file_name, 'objects'] = pd.Series(label)
            
            # Add people
            df.loc[df['filename'] == image_file_name, 'people'] = label.count('person')
            
            #Add "colorfulness" metric
            df.loc[df['filename'] == image_file_name,'colorfulness'] = image_colorfulness(image_array)
            
            # Add brightness and percieved brightness
            df.loc[df['filename'] == image_file_name, 'brightness'] = brightness(image_file_name)
            df.loc[df['filename'] == image_file_name, 'percieved_brightness'] = percieved_brightness(image_file_name)
            
            #Extract gender ratio of photo. 
            #gender_arary = detect_genders(faces, confidences, image_file_name)
            
            #if(len(faces) > 0):
                #df.loc[df['filename'] == image_file_name, 'percent_male'] = gender_arary.count('male')/float(len(faces))
                #df.loc[df['filename'] == image_file_name, 'percent_female'] = gender_arary.count('female')/float(len(faces))
            
        
                            
df = df.dropna()


In [92]:
print(df)

                                              filename num_faces  \
0    alexachung/57939604_1405275606280969_227927799...         1   
1    alexachung/57487966_302650897331937_5403985171...         0   
3    alexachung/58409673_1888377867934548_313100938...         0   
4    alexachung/57511927_446139556140698_1423052042...         1   
5    alexachung/58994690_801384873562945_4583796651...         0   
6    alexachung/57568586_144722209994647_2830766459...         0   
7    alexachung/58453539_318563102171701_5107544445...         1   
8    alexachung/57598467_109244776838216_8820137001...         1   
9    alexachung/59444228_294271814828332_6767495519...         0   
10   bucketlistjourney/59767226_464352457440775_604...         0   
11   bucketlistjourney/57303614_417134662419576_839...         0   
12   bucketlistjourney/56551834_425069051389781_109...         0   
13   bucketlistjourney/59121268_335244693724167_156...         0   
14   bucketlistjourney/57079872_417108132452805_

In [103]:
# LOG REGRESSION

train, test = train_test_split(df, test_size=0.2)

X_train = train[['num_faces', 'percieved_brightness', 
                           'colorfulness', 'people', 'brightness', 'people']]
y_train = np.ravel(train[['likes']])

X_test = test[['num_faces', 'percieved_brightness', 
                           'colorfulness', 'people', 'brightness', 'people']]
y_test = np.ravel(test[['likes']])

y_train = np.divide(y_train, 0.003)
y_train = y_train.astype(int)

y_test = np.divide(y_test, 0.003)
y_test = y_test.astype(int)

           
print(y_train)
#X_train, X_test, y_train, y_test = train_test_split(img_values, new_y, test_size=0.1, random_state=0)

clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial', max_iter=2000,
                         class_weight='balanced', verbose=5).fit(X_train, y_train)

y = clf.predict(X_test)

print(mean_squared_error(y_test, y))
print(y_test)
print(y)

[17  1  4  5 12  3  6  1  5  4  7  1  4  6  7  5  5  4  2 26 12 20  4 24
  2  1  8  7  1  1 30  1  7  1  7  4  3 21 10  0  5 32  6 31  2  4  8  6
  7  5  9  1 28 80  4  4 39 29 19 11 22  8 11  8]
1128.7058823529412
[20  7 20 17 71  0 14  8  1  1 46  6  3  5  5  6 26]
[30 80 24 39 11 31 80 39  2 30  1  9 24  1  1  2 22]


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s finished
